In [1]:
import requests
import requests_cache
requests_cache.install_cache('calrecycle')
import pandas as pd
import time

In [2]:
URL = 'https://www2.calrecycle.ca.gov/LGCentral/DisposalReporting/Destination/CountywideSummary'
params = {'CountyID': 58, 'ReportFormat': 'XLS'}
resp = requests.post(URL, data=params)
resp

<Response [200]>

In [3]:
import io

In [4]:
def set_columns(df, columns=None, row_idx=None):
    df = df.copy()
    if row_idx:
        columns = df.iloc[row_idx, :].tolist()
    df.columns = columns
    return df

In [5]:
(pd.read_excel(io.BytesIO(resp.content))
#  .iloc[4,:].tolist()
 .pipe(set_columns, row_idx=4)
 .iloc[5:, :]
 .dropna(axis=1, how='all')
 .assign(is_data_row=lambda d: d['Destination Facility'].notnull())
 .fillna(method='ffill')
 .query('is_data_row')
)

WARNING *** file size (47797) not 512 + multiple of sector size (512)


,Report Year,Quarter,Destination Facility,Diposal Ton,Total ADC,is_data_row
7,1995,1.0,Recology Yuba-Sutter Landfill,29594.00,NaN,True
8,1995,1.0,Yuba-Sutter Disposal Area,629.00,NaN,True
10,1995,2.0,Recology Yuba-Sutter Landfill,31012.00,NaN,True
11,1995,2.0,Yuba-Sutter Disposal Area,725.90,NaN,True
13,1995,3.0,Recology Ostrom Road LF Inc.,21288.00,NaN,True
14,1995,3.0,Recology Yuba-Sutter Landfill,11319.00,NaN,True
15,1995,3.0,Yuba-Sutter Disposal Area,830.00,NaN,True
17,1995,4.0,Recology Ostrom Road LF Inc.,6152.00,NaN,True
18,1995,4.0,Recology Yuba-Sutter Landfill,26269.00,NaN,True
21,1996,1.0,Recology Yuba-Sutter Landfill,32912.00,NaN,True


In [6]:
def make_throttle_hook(timeout=1):
    """
    Returns a response hook function which sleeps for `timeout` seconds if
    response is not coming from the cache.

    From https://requests-cache.readthedocs.io/en/latest/user_guide.html#usage
    """
    def hook(response, *args, **kwargs):
        if not getattr(response, 'from_cache', False):
            print(f'{response} not found in cache. Timeout for {timeout:.3f} s.')
            time.sleep(timeout)
        return response
    return hook

def get_session(rate_max=.5, timeout=None):
    
    timeout = 1 / rate_max

    s = requests_cache.CachedSession()
    s.hooks = {'response': make_throttle_hook(timeout)}
    return s

In [19]:
def process(df):
    return (df
             .pipe(set_columns, row_idx=4)
             .iloc[5:, :]
             .dropna(axis=1, how='all')
             .assign(is_data_row=lambda d: d['Destination Facility'].notnull())
             .fillna(method='ffill')
             .query('is_data_row')
             .drop(columns=['is_data_row'])
            )

def get_df(resp):
    if resp.ok:
        return pd.read_excel(io.BytesIO(resp.content))
    return pd.DataFrame()

# so ducky...
def get_report(county_id, session=requests):
    params = {'CountyID': int(county_id), 'ReportFormat': 'XLS'}
    # if "no record found", the server should return 404 instead of a 200 response with an empty XLS
    resp = session.post(URL, data=params)
    try:
        df = get_df(resp).pipe(process).assign(county_id=county_id)
    except Exception as e:
        print(e)
    else:
        return df

def get_reports():
    dfs = []
#     sesh = get_session(rate_max=2)
    ids = range(1, 58)
    for county_id in ids:
        df = get_report(county_id)
        if df is not None:
            dfs.append(df)
        else:
            print(f'county_id {county_id} not processed')
        # TODO else append to missed ids?
    return pd.concat(dfs)

def process_whole(df):
#     Destination Facility	Diposal Ton	Quarter	Report Year	Total ADC	Transformation Ton	county_id
    names = {
        'Destination Facility': 'destination_facility',
        'Diposal Ton': 'disposal',
        'Report Year': 'report_year',
        'Quarter': 'report_quarter',
        'Total ADC': 'total_adc',
        'Transformation Ton': 'transformation',
    }
    
    return (df
            .rename(columns=names)
            .fillna(0)
            .astype({'report_quarter': int})
           )

In [20]:
REPORTS = get_reports()

WARNING *** file size (83805) not 512 + multiple of sector size (512)
WARNING *** file size (9050) not 512 + multiple of sector size (512)
single positional indexer is out-of-bounds
county_id 2 not processed
WARNING *** file size (24704) not 512 + multiple of sector size (512)
WARNING *** file size (49724) not 512 + multiple of sector size (512)
WARNING *** file size (43747) not 512 + multiple of sector size (512)
WARNING *** file size (42316) not 512 + multiple of sector size (512)
WARNING *** file size (70569) not 512 + multiple of sector size (512)
WARNING *** file size (25583) not 512 + multiple of sector size (512)
WARNING *** file size (39402) not 512 + multiple of sector size (512)
WARNING *** file size (80939) not 512 + multiple of sector size (512)
WARNING *** file size (45953) not 512 + multiple of sector size (512)
WARNING *** file size (22621) not 512 + multiple of sector size (512)
WARNING *** file size (183139) not 512 + multiple of sector size (512)
WARNING *** file size

/home/ludo/miniconda3/envs/catdd/lib/python3.6/site-packages/ipykernel_launcher.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [25]:
REPORTS = REPORTS.pipe(process_whole)
REPORTS

,destination_facility,disposal,report_quarter,report_year,total_adc,transformation,county_id
7,Altamont Landfill & Resource Recovery,396727.00,1,1995,174.00,0.0,1
8,Tri Cities Recycling & Disposal Fac,67410.00,1,1995,174.00,0.0,1
9,Vasco Road Sanitary Landfill,111209.00,1,1995,174.00,0.0,1
11,Altamont Landfill & Resource Recovery,393888.00,2,1995,16991.00,0.0,1
12,Tri Cities Recycling & Disposal Fac,72408.00,2,1995,4379.00,0.0,1
13,Vasco Road Sanitary Landfill,114093.00,2,1995,4379.00,0.0,1
15,Altamont Landfill & Resource Recovery,394990.00,3,1995,18181.00,0.0,1
16,Tri Cities Recycling & Disposal Fac,73940.00,3,1995,9951.00,0.0,1
17,Vasco Road Sanitary Landfill,122358.00,3,1995,9951.00,0.0,1
19,Altamont Landfill & Resource Recovery,383876.00,4,1995,23069.00,0.0,1


In [24]:
REPORTS.to_csv('/data/datasets/catdd/clean/calrecycle-disposal-reporting.csv')